In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from bs4 import BeautifulSoup 
import requests

from RGGI_plant_analysis import RGGI_capacity

In [2]:
import sys
from statistics import NormalDist
import subprocess
import os 

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

from enviro_vars import set_environmental_variables
try:       
    set_environmental_variables()
except Exception as e:
    print(e)
    print('in online environment - no enviro_vars module')

############# ADD GIT TOKEN BELOW ############
git_token = os.environ['gittoken']
AUTHORITY = os.environ['azAuth']
CLIENT_ID = os.environ['azClientid']
SECRET = os.environ['AzAppSecret']
location =  {"authority": AUTHORITY,"client_id": CLIENT_ID,'secret':SECRET,"scope": [ "https://graph.microsoft.com/.default" ]}

##
github_repository = f"git+https://{git_token}@github.com/research-carbon-cap/SharePointv2.git@Azure_functions"

install(github_repository)


from SharePointv2.Sharepoint_API import GRAPH_API

Environment variables set!


In [3]:
next_update_time,recent_report,report_month,report_year,date_of_last_report  =RGGI_capacity(location).scrape_recent_EIA_860m(lagged_report=1)

saving down data for 2024-10-01
Operating
Planned
Retired
Canceled or Postponed
Operating_PR
Planned_PR
Retired_PR


In [15]:
next_update_time

Timestamp('2024-12-23 00:00:00')

In [17]:
report_year

2024

In [4]:
plants = pd.read_excel(recent_report,sheet_name='Operating')
header_col = plants[(plants.iloc[:,0]=='Entity ID')==True].index[0]
plants = pd.read_excel(recent_report,sheet_name='Operating',header=header_col+1)
plants['report_month'] = report_month
plants['report_year'] = report_year
plants = plants.dropna(subset=['Plant Name'])
RGGI_plants =  plants.query(f'`Plant State`=={RGGI_capacity(location).RGGI_states}') # or `Balancing Authority Code`=="PJM" 
full_tech_list = plants.Technology.unique()

RGGI_plants = RGGI_capacity(location).analyse_RGGI_capacity(RGGI_plants)

c:\GitHub\RGGI_capacity_data\RGGI_plant_analysis.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RGGI_plants['Fossil'] =  np.where(RGGI_plants['Technology'].isin(self.Fossil_tech),1,0)
c:\GitHub\RGGI_capacity_data\RGGI_plant_analysis.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RGGI_plants['PlantName_GenID'] = RGGI_plants['Plant Name'] +' '+ RGGI_plants['Generator ID']
c:\GitHub\RGGI_capacity_data\RGGI_plant_analysis.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [6]:
planned = pd.read_excel(recent_report,sheet_name='Planned',header=2)
RGGI_planned = planned.query(f'`Plant State`=={RGGI_capacity(location).RGGI_states} ') #or `Balancing Authority Code`=="PJM"
additions,approved_additions,not_yet_approved_additions = RGGI_capacity(location).analyse_RGGI_planned_capacity(RGGI_planned)

c:\GitHub\RGGI_capacity_data\RGGI_plant_analysis.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RGGI_planned[['year','month']] = RGGI_planned[['Planned Operation Year','Planned Operation Month']]
c:\GitHub\RGGI_capacity_data\RGGI_plant_analysis.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RGGI_planned[['year','month']] = RGGI_planned[['Planned Operation Year','Planned Operation Month']]
c:\GitHub\RGGI_capacity_data\RGGI_plant_analysis.py:129: SettingWithCopyWarning: 
A value is trying to be 

In [20]:
px.pie(RGGI_plants.groupby('Technology')['Nameplate Capacity (MW)'].sum())

In [7]:
planned_fossil_retirments_fig,all_planned_capacity_fig,approved_capacity_fig, not_yet_approved_capacity_fig,RE_total_vs_approved_fig,cumulative_retirements_fig,rggi_capacity_by_tech_fig = RGGI_capacity(location).RGGI_capacity_charts(RGGI_plants,additions,approved_additions,not_yet_approved_additions)

In [8]:
rggi_capacity_by_tech_fig

In [9]:
planned_fossil_retirments_fig

In [21]:
cumulative_retirements_fig

In [10]:
all_planned_capacity_fig

In [11]:
approved_capacity_fig

In [12]:
not_yet_approved_capacity_fig

In [22]:
RE_total_vs_approved_fig

In [13]:
rggi_plus_retirements_plus_additions_timeseries, time_series_capacity_fig = RGGI_capacity(location).estimated_timeseries_capacity(RGGI_plants,full_tech_list,additions,date_of_last_report,PJM_retiredates=True)

In [33]:
(rggi_plus_retirements_plus_additions_timeseries.sum(axis=1).pct_change()+1).iloc[2:].cumprod()

2024-11-01    1.002195
2024-12-01    1.004629
2024-12-01    1.007178
2025-01-01    1.007729
2025-02-01    1.008914
                ...   
2030-09-01    1.102775
2030-10-01    1.102775
2030-11-01    1.102775
2030-12-01    1.102775
2039-10-01    1.102740
Length: 83, dtype: float64

In [ ]:

time_series_capacity_fig.update_layout(width=1000,height=500)

In [36]:
plants['Balancing Authority Code'].unique()

array([nan, 'SOCO', 'EPE', 'MISO', 'CISO', 'TVA', 'PJM', 'SWPP', 'NYIS',
       'PSEI', 'SRP', 'AZPS', 'TEPC', 'SC', 'SEC', 'BANC', 'WALC', 'PACE',
       'TIDC', 'SPA', 'PACW', 'ERCO', 'IID', 'LDWP', 'BPAT', 'PSCO',
       'WACM', 'WAUW', 'ISNE', 'AVA', 'FMPP', 'FPL', 'SCL', 'FPC', 'TEC',
       'GVL', 'HST', 'JEA', 'TAL', 'SCEG', 'SEPA', 'IPCO', 'GCPD', 'CPLE',
       'AECI', 'LGEE', 'NBSO', 'NWMT', 'NEVP', 'PNM', 'DUK', 'PGE',
       'CHPD', 'DOPD', 'TPWR', 'YAD', 'AVRN', 'GRIF', 'DEAA', 'GRID',
       'HGMA', 'HECO', 'GWA', 'WWA', 'GRIS', 'CPLW'], dtype=object)

In [38]:
def get_full_retirements_additions_per_iso(plants,planned,full_tech_list,date_of_last_report, ISO): ## PJM, ISNE, NYIS
    iso_plants = plants[plants['Balancing Authority Code']==ISO]
    iso_plants_analysed = RGGI_capacity(location).analyse_RGGI_capacity(iso_plants)
    ISO_planned = planned[planned['Balancing Authority Code']==ISO]
    additions,approved_additions,not_yet_approved_additions = RGGI_capacity(location).analyse_RGGI_planned_capacity(ISO_planned)
    iso_plus_retirements_plus_additions_timeseries, iso_time_series_capacity_fig = RGGI_capacity(location).estimated_timeseries_capacity(iso_plants_analysed,full_tech_list,additions,date_of_last_report,PJM_retiredates=True)
    
    return iso_plus_retirements_plus_additions_timeseries, iso_time_series_capacity_fig

In [39]:
pjm_plus_retirements_plus_additions_timeseries, pjm_time_series_capacity_fig = get_full_retirements_additions_per_iso(plants,planned,full_tech_list,date_of_last_report, ISO='PJM')
isne_plus_retirements_plus_additions_timeseries, isne_time_series_capacity_fig = get_full_retirements_additions_per_iso(plants,planned,full_tech_list,date_of_last_report, ISO='ISNE')
nyis_plus_retirements_plus_additions_timeseries, nyis_time_series_capacity_fig = get_full_retirements_additions_per_iso(plants,planned,full_tech_list,date_of_last_report, ISO='NYIS')

c:\GitHub\RGGI_capacity_data\RGGI_plant_analysis.py:103: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\GitHub\RGGI_capacity_data\RGGI_plant_analysis.py:104: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\GitHub\RGGI_capacity_data\RGGI_plant_analysis.py:107: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/index

In [ ]:
isne_plus_retirements_plus_additions_timeseries.

,All Other,Batteries,Coal Integrated Gasification Combined Cycle,Conventional Hydroelectric,Conventional Steam Coal,Flywheels,Geothermal,Hydroelectric Pumped Storage,Landfill Gas,Municipal Solid Waste,...,Onshore Wind Turbine,Other Gases,Other Natural Gas,Other Waste Biomass,Petroleum Coke,Petroleum Liquids,Solar Photovoltaic,Solar Thermal with Energy Storage,Solar Thermal without Energy Storage,Wood/Wood Waste Biomass
2024-10-01,0.0,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0
2024-10-01,0.0,388.5,0.0,1919.6,559.2,0.0,0.0,1865.0,98.7,464.0,...,1510.2,0.0,111.1,20.7,0.0,6587.3,3021.2,0.0,0.0,832.8
2024-11-01,0.0,391.5,0.0,1919.6,559.2,0.0,0.0,1865.0,98.7,464.0,...,1510.2,0.0,111.1,20.7,0.0,6587.3,3209.5,0.0,0.0,832.8
2024-12-01,0.0,403.8,0.0,1919.6,559.2,0.0,0.0,1865.0,98.7,464.0,...,1510.2,0.0,111.1,20.7,0.0,6574.1,3312.8,0.0,0.0,832.8
2024-12-01,0.0,416.1,0.0,1919.6,559.2,0.0,0.0,1865.0,98.7,464.0,...,1510.2,0.0,111.1,20.7,0.0,6574.1,3416.1,0.0,0.0,832.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2030-09-01,0.0,1192.6,0.0,1919.6,559.2,0.0,0.0,1865.0,98.7,464.0,...,1569.0,0.0,111.1,20.7,0.0,6568.1,4019.7,0.0,0.0,832.8
2030-10-01,0.0,1192.6,0.0,1919.6,559.2,0.0,0.0,1865.0,98.7,464.0,...,1569.0,0.0,111.1,20.7,0.0,6568.1,4019.7,0.0,0.0,832.8
2030-11-01,0.0,1192.6,0.0,1919.6,559.2,0.0,0.0,1865.0,98.7,464.0,...,1569.0,0.0,111.1,20.7,0.0,6568.1,4019.7,0.0,0.0,832.8
2030-12-01,0.0,1192.6,0.0,1919.6,559.2,0.0,0.0,1865.0,98.7,464.0,...,1569.0,0.0,111.1,20.7,0.0,6568.1,4019.7,0.0,0.0,832.8


In [40]:
nyis_time_series_capacity_fig

In [46]:
isne_time_series_capacity_fig.update_layout(width=800)

In [42]:
pjm_time_series_capacity_fig

In [45]:
plants.query('`Plant ID`==65197').iloc[0]

Entity ID                                                   57128
Entity Name                                          Bloom Energy
Plant ID                                                  65197.0
Plant Name                                Medtronic - New Haven 2
Google Map                                                    Map
Bing Map                                                      Map
Plant State                                                    CT
County                                                  New Haven
Balancing Authority Code                                     ISNE
Sector                                                IPP Non-CHP
Generator ID                                                MDC04
Unit Code                                                     NaN
Nameplate Capacity (MW)                                       2.2
Net Summer Capacity (MW)                                      2.2
Net Winter Capacity (MW)                                      2.2
Technology